In [13]:
"""
PROJET AVEC GRAPHES COMPLETS - ANALYSE DES VENTES DE VÉLOS
Dataset: 113,000 transactions
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 1. CHARGEMENT ET PRÉPARATION DES DONNÉES
# ============================================================================

print("="*80)
print("🚴 ANALYSE DES VENTES DE VÉLOS AVEC GRAPHES COMPLETS")
print("="*80)

# Chargement des données
file_path = "data/sales_data.csv"
df = pd.read_csv(file_path, encoding='utf-16', header=None)
df = df[0].str.split(',', expand=True)
df.columns = df.iloc[0]
df = df[1:]

if df.columns[-1] is None:
    df = df.iloc[:, :-1]

df.columns = df.columns.str.strip().str.replace('\ufeff','').str.replace('"','')

# Nettoyage des colonnes numériques
numeric_cols = ["Customer_Age","Order_Quantity","Unit_Cost","Unit_Price","Profit","Cost","Revenue"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col].str.strip().str.replace('"',''), errors='coerce')

# Conversion des dates
df['Date'] = pd.to_datetime(df['Date'].str.strip().str.replace('"',''))

print(f"✅ Données chargées : {len(df):,} transactions")
print(f"📅 Période : {df['Date'].min().date()} au {df['Date'].max().date()}")

# Préparation des données temporelles
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month_Name'] = df['Date'].dt.month_name()
df['Quarter'] = df['Date'].dt.quarter
df['Week'] = df['Date'].dt.isocalendar().week
df['Day_Of_Week'] = df['Date'].dt.day_name()
df['Day_Of_Year'] = df['Date'].dt.dayofyear

# Conversion des mois en français
mois_fr = {
    'January': 'Janvier', 'February': 'Février', 'March': 'Mars',
    'April': 'Avril', 'May': 'Mai', 'June': 'Juin',
    'July': 'Juillet', 'August': 'Août', 'September': 'Septembre',
    'October': 'Octobre', 'November': 'Novembre', 'December': 'Décembre'
}
df['Month_Name_FR'] = df['Month_Name'].map(mois_fr)

# Calcul des métriques
df['Margin_Pct'] = (df['Profit'] / df['Revenue'] * 100).round(2)
df['Unit_Margin'] = df['Unit_Price'] - df['Unit_Cost']
df['Transaction_Value'] = df['Revenue'] / df['Order_Quantity']

# Saisons
def get_season(month):
    if month in [12, 1, 2]:
        return 'Hiver'
    elif month in [3, 4, 5]:
        return 'Printemps'
    elif month in [6, 7, 8]:
        return 'Été'
    else:
        return 'Automne'

df['Season'] = df['Month'].apply(get_season)

# ============================================================================
# 2. GRAPHES : ANALYSE TEMPORELLE
# ============================================================================

print("\n📈 CRÉATION DES GRAPHES TEMPORELS...")

# Configuration des styles
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Figure 1: Évolution des ventes dans le temps
fig1, axes1 = plt.subplots(2, 2, figsize=(16, 10))

# 1.1 Revenue mensuel
monthly_revenue = df.groupby(['Year', 'Month', 'Month_Name_FR'])['Revenue'].sum().reset_index()
monthly_revenue['Date'] = pd.to_datetime(monthly_revenue['Year'].astype(str) + '-' + 
                                         monthly_revenue['Month'].astype(str) + '-01')

axes1[0, 0].plot(monthly_revenue['Date'], monthly_revenue['Revenue'], 
                marker='o', linewidth=2, color='steelblue')
axes1[0, 0].set_title('Évolution du Revenue Mensuel', fontsize=14, fontweight='bold')
axes1[0, 0].set_xlabel('Date')
axes1[0, 0].set_ylabel('Revenue (€)')
axes1[0, 0].grid(True, alpha=0.3)
axes1[0, 0].tick_params(axis='x', rotation=45)

# 1.2 Revenue par mois (toutes années)
revenue_by_month = df.groupby(['Month', 'Month_Name_FR'])['Revenue'].sum().reset_index()
revenue_by_month = revenue_by_month.sort_values('Month')

bars = axes1[0, 1].bar(revenue_by_month['Month_Name_FR'], revenue_by_month['Revenue'],
                       color='coral', alpha=0.7)
axes1[0, 1].set_title('Revenue Total par Mois', fontsize=14, fontweight='bold')
axes1[0, 1].set_xlabel('Mois')
axes1[0, 1].set_ylabel('Revenue (€)')
axes1[0, 1].tick_params(axis='x', rotation=45)
axes1[0, 1].grid(True, alpha=0.3, axis='y')

# Ajouter les valeurs sur les barres
for bar in bars:
    height = bar.get_height()
    axes1[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:,.0f}€', ha='center', va='bottom', fontsize=9)

# 1.3 Revenue par jour de la semaine
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['Day_Of_Week'] = pd.Categorical(df['Day_Of_Week'], categories=weekday_order, ordered=True)
revenue_by_weekday = df.groupby('Day_Of_Week')['Revenue'].sum()

jours_fr = {'Monday': 'Lundi', 'Tuesday': 'Mardi', 'Wednesday': 'Mercredi',
            'Thursday': 'Jeudi', 'Friday': 'Vendredi', 'Saturday': 'Samedi',
            'Sunday': 'Dimanche'}

revenue_by_weekday.index = revenue_by_weekday.index.map(jours_fr)

axes1[1, 0].bar(revenue_by_weekday.index, revenue_by_weekday.values,
                color='mediumseagreen', alpha=0.7)
axes1[1, 0].set_title('Revenue par Jour de la Semaine', fontsize=14, fontweight='bold')
axes1[1, 0].set_xlabel('Jour')
axes1[1, 0].set_ylabel('Revenue (€)')
axes1[1, 0].tick_params(axis='x', rotation=45)
axes1[1, 0].grid(True, alpha=0.3, axis='y')

# 1.4 Revenue par saison
revenue_by_season = df.groupby('Season')['Revenue'].sum()
season_order = ['Printemps', 'Été', 'Automne', 'Hiver']
revenue_by_season = revenue_by_season.reindex(season_order)

axes1[1, 1].pie(revenue_by_season.values, labels=revenue_by_season.index,
               autopct='%1.1f%%', startangle=90, colors=['#4CAF50', '#FF9800', '#795548', '#2196F3'])
axes1[1, 1].set_title('Répartition des Ventes par Saison', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('graphe_temporal.png', dpi=300, bbox_inches='tight')
print("✅ Graphique temporel sauvegardé: 'graphe_temporal.png'")

# ============================================================================
# 3. GRAPHES : ANALYSE DES CATÉGORIES
# ============================================================================

print("\n📦 CRÉATION DES GRAPHES PAR CATÉGORIE...")

fig2, axes2 = plt.subplots(2, 2, figsize=(16, 10))

# 2.1 Revenue par catégorie de produit
if 'Product_Category' in df.columns:
    revenue_by_category = df.groupby('Product_Category')['Revenue'].sum().sort_values(ascending=False)
    
    bars = axes2[0, 0].bar(revenue_by_category.index, revenue_by_category.values,
                          color='lightcoral', alpha=0.7)
    axes2[0, 0].set_title('Revenue par Catégorie de Produit', fontsize=14, fontweight='bold')
    axes2[0, 0].set_xlabel('Catégorie')
    axes2[0, 0].set_ylabel('Revenue (€)')
    axes2[0, 0].tick_params(axis='x', rotation=45)
    axes2[0, 0].grid(True, alpha=0.3, axis='y')
    
    # Ajouter les pourcentages
    total_revenue = revenue_by_category.sum()
    for i, (category, revenue) in enumerate(revenue_by_category.items()):
        percentage = (revenue / total_revenue * 100)
        axes2[0, 0].text(i, revenue, f'{percentage:.1f}%', 
                        ha='center', va='bottom', fontsize=9)

# 2.2 Marge par catégorie
if 'Product_Category' in df.columns:
    margin_by_category = df.groupby('Product_Category')['Margin_Pct'].mean().sort_values(ascending=False)
    
    colors = ['green' if x > margin_by_category.mean() else 'red' for x in margin_by_category.values]
    bars = axes2[0, 1].bar(margin_by_category.index, margin_by_category.values,
                          color=colors, alpha=0.7)
    axes2[0, 1].axhline(y=margin_by_category.mean(), color='blue', linestyle='--',
                       label=f'Moyenne: {margin_by_category.mean():.1f}%')
    axes2[0, 1].set_title('Marge Moyenne par Catégorie', fontsize=14, fontweight='bold')
    axes2[0, 1].set_xlabel('Catégorie')
    axes2[0, 1].set_ylabel('Marge (%)')
    axes2[0, 1].tick_params(axis='x', rotation=45)
    axes2[0, 1].legend()
    axes2[0, 1].grid(True, alpha=0.3, axis='y')

# 2.3 Top 10 produits
if 'Product' in df.columns:
    top_products = df.groupby('Product')['Revenue'].sum().nlargest(10).sort_values()
    
    bars = axes2[1, 0].barh(range(len(top_products)), top_products.values,
                           color='goldenrod', alpha=0.7)
    axes2[1, 0].set_yticks(range(len(top_products)))
    axes2[1, 0].set_yticklabels(top_products.index)
    axes2[1, 0].set_title('Top 10 Produits par Revenue', fontsize=14, fontweight='bold')
    axes2[1, 0].set_xlabel('Revenue (€)')
    axes2[1, 0].grid(True, alpha=0.3, axis='x')
    
    # Ajouter les valeurs
    for i, (product, revenue) in enumerate(top_products.items()):
        axes2[1, 0].text(revenue, i, f' {revenue:,.0f}€', 
                        va='center', fontsize=9)

# 2.4 Quantité vendue par catégorie
if 'Product_Category' in df.columns and 'Order_Quantity' in df.columns:
    quantity_by_category = df.groupby('Product_Category')['Order_Quantity'].sum().sort_values(ascending=False)
    
    axes2[1, 1].pie(quantity_by_category.values, labels=quantity_by_category.index,
                   autopct='%1.1f%%', startangle=90)
    axes2[1, 1].set_title('Quantité Vendue par Catégorie', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('graphe_categories.png', dpi=300, bbox_inches='tight')
print("✅ Graphique catégories sauvegardé: 'graphe_categories.png'")

# ============================================================================
# 4. GRAPHES : ANALYSE DES CLIENTS
# ============================================================================

print("\n👥 CRÉATION DES GRAPHES CLIENTS...")

fig3, axes3 = plt.subplots(2, 2, figsize=(16, 10))

# 3.1 Distribution des âges
if 'Customer_Age' in df.columns:
    axes3[0, 0].hist(df['Customer_Age'].dropna(), bins=20, color='skyblue', 
                    edgecolor='black', alpha=0.7)
    axes3[0, 0].axvline(x=df['Customer_Age'].mean(), color='red', linestyle='--',
                       label=f'Moyenne: {df["Customer_Age"].mean():.1f} ans')
    axes3[0, 0].set_title('Distribution de l\'Âge des Clients', fontsize=14, fontweight='bold')
    axes3[0, 0].set_xlabel('Âge')
    axes3[0, 0].set_ylabel('Nombre de Clients')
    axes3[0, 0].legend()
    axes3[0, 0].grid(True, alpha=0.3)

# 3.2 Revenue par groupe d'âge
if 'Customer_Age' in df.columns:
    df['Age_Group'] = pd.cut(df['Customer_Age'], 
                            bins=[0, 25, 35, 50, 65, 100],
                            labels=['<25', '25-34', '35-49', '50-64', '65+'])
    
    revenue_by_age = df.groupby('Age_Group')['Revenue'].sum()
    
    bars = axes3[0, 1].bar(revenue_by_age.index, revenue_by_age.values,
                          color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
    axes3[0, 1].set_title('Revenue par Groupe d\'Âge', fontsize=14, fontweight='bold')
    axes3[0, 1].set_xlabel('Groupe d\'Âge')
    axes3[0, 1].set_ylabel('Revenue (€)')
    axes3[0, 1].grid(True, alpha=0.3, axis='y')
    
    for bar in bars:
        height = bar.get_height()
        axes3[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:,.0f}€', ha='center', va='bottom', fontsize=9)

# 3.3 Distribution par genre
if 'Customer_Gender' in df.columns:
    gender_dist = df['Customer_Gender'].value_counts()
    
    axes3[1, 0].pie(gender_dist.values, labels=gender_dist.index,
                   autopct='%1.1f%%', startangle=90, colors=['lightblue', 'lightpink'])
    axes3[1, 0].set_title('Répartition par Genre', fontsize=14, fontweight='bold')

# 3.4 Panier moyen par groupe d'âge
if 'Customer_Age' in df.columns:
    basket_by_age = df.groupby('Age_Group')['Transaction_Value'].mean()
    
    axes3[1, 1].plot(basket_by_age.index, basket_by_age.values, 
                    marker='o', linewidth=2, color='purple')
    axes3[1, 1].set_title('Panier Moyen par Groupe d\'Âge', fontsize=14, fontweight='bold')
    axes3[1, 1].set_xlabel('Groupe d\'Âge')
    axes3[1, 1].set_ylabel('Panier Moyen (€)')
    axes3[1, 1].grid(True, alpha=0.3)
    
    for i, (age_group, value) in enumerate(basket_by_age.items()):
        axes3[1, 1].text(i, value, f'{value:.0f}€', 
                        ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('graphe_clients.png', dpi=300, bbox_inches='tight')
print("✅ Graphique clients sauvegardé: 'graphe_clients.png'")

# ============================================================================
# 5. GRAPHES : ANALYSE GÉOGRAPHIQUE ET MÉTRIQUES
# ============================================================================

print("\n🌍 CRÉATION DES GRAPHES GÉOGRAPHIQUES...")

fig4, axes4 = plt.subplots(2, 2, figsize=(16, 10))

# 5.1 Revenue par pays
if 'Country' in df.columns:
    revenue_by_country = df.groupby('Country')['Revenue'].sum().sort_values(ascending=False).head(10)
    
    bars = axes4[0, 0].bar(revenue_by_country.index, revenue_by_country.values,
                          color='steelblue', alpha=0.7)
    axes4[0, 0].set_title('Top 10 Pays par Revenue', fontsize=14, fontweight='bold')
    axes4[0, 0].set_xlabel('Pays')
    axes4[0, 0].set_ylabel('Revenue (€)')
    axes4[0, 0].tick_params(axis='x', rotation=45)
    axes4[0, 0].grid(True, alpha=0.3, axis='y')
    
    for bar in bars:
        height = bar.get_height()
        axes4[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:,.0f}€', ha='center', va='bottom', fontsize=9)

# 5.2 Heatmap: Revenue par mois et année
heatmap_data = df.groupby(['Year', 'Month'])['Revenue'].sum().unstack()
heatmap_data = heatmap_data.reindex(columns=range(1, 13))

sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='YlOrRd', 
            cbar_kws={'label': 'Revenue (€)'}, ax=axes4[0, 1])
axes4[0, 1].set_title('Heatmap: Revenue par Mois et Année', fontsize=14, fontweight='bold')
axes4[0, 1].set_xlabel('Mois')
axes4[0, 1].set_ylabel('Année')

# 5.3 Évolution de la marge dans le temps
monthly_margin = df.groupby(['Year', 'Month'])['Margin_Pct'].mean().reset_index()
monthly_margin['Date'] = pd.to_datetime(monthly_margin['Year'].astype(str) + '-' + 
                                       monthly_margin['Month'].astype(str) + '-01')

axes4[1, 0].plot(monthly_margin['Date'], monthly_margin['Margin_Pct'], 
                marker='o', linewidth=2, color='green')
axes4[1, 0].axhline(y=monthly_margin['Margin_Pct'].mean(), color='red', linestyle='--',
                   label=f'Moyenne: {monthly_margin["Margin_Pct"].mean():.1f}%')
axes4[1, 0].set_title('Évolution de la Marge Mensuelle', fontsize=14, fontweight='bold')
axes4[1, 0].set_xlabel('Date')
axes4[1, 0].set_ylabel('Marge (%)')
axes4[1, 0].tick_params(axis='x', rotation=45)
axes4[1, 0].legend()
axes4[1, 0].grid(True, alpha=0.3)

# 5.4 Distribution du panier moyen
daily_basket = df.groupby('Date')['Transaction_Value'].mean()

axes4[1, 1].hist(daily_basket.dropna(), bins=30, color='orange', 
                edgecolor='black', alpha=0.7)
axes4[1, 1].axvline(x=daily_basket.mean(), color='red', linestyle='--',
                   label=f'Moyenne: {daily_basket.mean():.1f}€')
axes4[1, 1].set_title('Distribution du Panier Moyen Journalier', fontsize=14, fontweight='bold')
axes4[1, 1].set_xlabel('Panier Moyen (€)')
axes4[1, 1].set_ylabel('Fréquence')
axes4[1, 1].legend()
axes4[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('graphe_geo_metriques.png', dpi=300, bbox_inches='tight')
print("✅ Graphique géo & métriques sauvegardé: 'graphe_geo_metriques.png'")

# ============================================================================
# 6. GRAPHES : DASHBOARD SYNTHÈSE
# ============================================================================

print("\n📊 CRÉATION DU DASHBOARD SYNTHÈSE...")

# Figure synthèse avec les KPIs principaux
fig5 = plt.figure(figsize=(18, 12))

# Calcul des KPIs
total_revenue = df['Revenue'].sum()
total_profit = df['Profit'].sum()
total_transactions = len(df)
avg_basket = df['Revenue'].sum() / total_transactions
margin_rate = (total_profit / total_revenue * 100) if total_revenue > 0 else 0
avg_daily_revenue = df.groupby('Date')['Revenue'].sum().mean()

# 1. KPI Cards (simulées avec subplots)
ax1 = plt.subplot(3, 4, 1)
ax1.text(0.5, 0.7, f"{total_revenue:,.0f}€", ha='center', va='center', 
        fontsize=24, fontweight='bold', color='steelblue')
ax1.text(0.5, 0.4, "Revenue Total", ha='center', va='center', 
        fontsize=14, style='italic')
ax1.axis('off')
ax1.set_facecolor('#f0f0f0')

ax2 = plt.subplot(3, 4, 2)
ax2.text(0.5, 0.7, f"{total_profit:,.0f}€", ha='center', va='center', 
        fontsize=24, fontweight='bold', color='green')
ax2.text(0.5, 0.4, "Profit Total", ha='center', va='center', 
        fontsize=14, style='italic')
ax2.axis('off')
ax2.set_facecolor('#f0f0f0')

ax3 = plt.subplot(3, 4, 3)
ax3.text(0.5, 0.7, f"{margin_rate:.1f}%", ha='center', va='center', 
        fontsize=24, fontweight='bold', color='coral')
ax3.text(0.5, 0.4, "Marge Brute", ha='center', va='center', 
        fontsize=14, style='italic')
ax3.axis('off')
ax3.set_facecolor('#f0f0f0')

ax4 = plt.subplot(3, 4, 4)
ax4.text(0.5, 0.7, f"{avg_basket:.0f}€", ha='center', va='center', 
        fontsize=24, fontweight='bold', color='purple')
ax4.text(0.5, 0.4, "Panier Moyen", ha='center', va='center', 
        fontsize=14, style='italic')
ax4.axis('off')
ax4.set_facecolor('#f0f0f0')

# 2. Tendances mensuelles
ax5 = plt.subplot(3, 2, 3)
monthly_revenue_all = df.groupby(['Year', 'Month'])['Revenue'].sum().reset_index()
for year in monthly_revenue_all['Year'].unique():
    year_data = monthly_revenue_all[monthly_revenue_all['Year'] == year]
    ax5.plot(year_data['Month'], year_data['Revenue'], marker='o', label=str(year))
ax5.set_title('Tendances Mensuelles par Année', fontsize=14, fontweight='bold')
ax5.set_xlabel('Mois')
ax5.set_ylabel('Revenue (€)')
ax5.legend(title='Année')
ax5.grid(True, alpha=0.3)
ax5.set_xticks(range(1, 13))

# 3. Saisonnalité
ax6 = plt.subplot(3, 2, 4)
revenue_season = df.groupby(['Season', 'Month'])['Revenue'].mean().unstack('Season')
revenue_season = revenue_season.reindex(columns=['Printemps', 'Été', 'Automne', 'Hiver'])
revenue_season.plot(kind='bar', ax=ax6)
ax6.set_title('Saisonnalité des Ventes', fontsize=14, fontweight='bold')
ax6.set_xlabel('Mois')
ax6.set_ylabel('Revenue Moyen (€)')
ax6.legend(title='Saison')
ax6.grid(True, alpha=0.3, axis='y')
ax6.tick_params(axis='x', rotation=0)

# 4. Top catégories
ax7 = plt.subplot(3, 2, 5)
if 'Product_Category' in df.columns:
    top_cats = df.groupby('Product_Category')['Revenue'].sum().nlargest(5)
    colors = plt.cm.Set3(np.linspace(0, 1, len(top_cats)))
    ax7.pie(top_cats.values, labels=top_cats.index, colors=colors,
           autopct='%1.1f%%', startangle=90)
    ax7.set_title('Top 5 Catégories', fontsize=14, fontweight='bold')

# 5. Distribution temporelle
ax8 = plt.subplot(3, 2, 6)
hourly_pattern = df.groupby(df['Date'].dt.hour)['Revenue'].sum()
ax8.bar(hourly_pattern.index, hourly_pattern.values, color='teal', alpha=0.7)
ax8.set_title('Distribution Horaire des Ventes', fontsize=14, fontweight='bold')
ax8.set_xlabel('Heure de la Journée')
ax8.set_ylabel('Revenue (€)')
ax8.grid(True, alpha=0.3, axis='y')

plt.suptitle('DASHBOARD SYNTHÈSE - ANALYSE DES VENTES DE VÉLOS', 
            fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('dashboard_synthese.png', dpi=300, bbox_inches='tight')
print("✅ Dashboard synthèse sauvegardé: 'dashboard_synthese.png'")

# ============================================================================
# 7. AFFICHAGE ET EXPORT
# ============================================================================

print("\n" + "="*80)
print("📊 RÉSUMÉ DES GRAPHES CRÉÉS")
print("="*80)

print("\n✅ GRAPHES SAUVEGARDÉS :")
print("   1. graphe_temporal.png      - Analyses temporelles")
print("   2. graphe_categories.png    - Analyses par catégorie")
print("   3. graphe_clients.png       - Analyses clients")
print("   4. graphe_geo_metriques.png - Analyses géo & métriques")
print("   5. dashboard_synthese.png   - Dashboard complet")

print(f"\n📈 KPIs PRINCIPAUX :")
print(f"   • Revenue total       : {total_revenue:,.0f}€")
print(f"   • Profit total        : {total_profit:,.0f}€")
print(f"   • Marge brute         : {margin_rate:.1f}%")
print(f"   • Panier moyen        : {avg_basket:.0f}€")
print(f"   • Transactions        : {total_transactions:,}")
print(f"   • Revenue/jour moyen  : {avg_daily_revenue:,.0f}€")

print(f"\n📅 PÉRIODE ANALYSÉE :")
print(f"   • Début : {df['Date'].min().date()}")
print(f"   • Fin   : {df['Date'].max().date()}")
print(f"   • Durée : {(df['Date'].max() - df['Date'].min()).days} jours")

print(f"\n🌍 DONNÉES GÉOGRAPHIQUES :")
if 'Country' in df.columns:
    countries = df['Country'].nunique()
    print(f"   • Pays couverts     : {countries}")
    top_country = df.groupby('Country')['Revenue'].sum().idxmax()
    top_country_rev = df.groupby('Country')['Revenue'].sum().max()
    print(f"   • Meilleur pays     : {top_country} ({top_country_rev:,.0f}€)")

print(f"\n📦 ANALYSE PRODUITS :")
if 'Product_Category' in df.columns:
    categories = df['Product_Category'].nunique()
    print(f"   • Catégories        : {categories}")
    top_category = df.groupby('Product_Category')['Revenue'].sum().idxmax()
    print(f"   • Meilleure catégorie : {top_category}")

print("\n" + "="*80)
print("🎯 ANALYSE TERMINÉE !")
print("Ouvrez les fichiers PNG pour visualiser les graphiques.")
print("="*80)

# Afficher tous les graphiques
plt.show()

🚴 ANALYSE DES VENTES DE VÉLOS AVEC GRAPHES COMPLETS


FileNotFoundError: [Errno 2] No such file or directory: 'data/sales_data.csv'

In [14]:
print('fin du script')

fin du script
